# Phase 2: Feature Engineering

**Regime-Switching LSTM for SPY Directional Prediction**

This notebook engineers all features needed for the regime-switching LSTM model, including:
- Lag features (momentum/mean reversion)
- Rolling statistics and price ratios
- VIX features and regime indicators
- Regime transition features
- Volume features
- Target variable (5-day forward direction)

**Output**: `data/features_engineered.csv` ready for modeling

## 1. Setup & Data Loading

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import joblib
import warnings
warnings.filterwarnings('ignore')

# Set paths
BASE_DIR = os.path.dirname(os.getcwd())
DATA_DIR = os.path.join(BASE_DIR, 'data')
MODELS_DIR = os.path.join(BASE_DIR, 'models')
os.makedirs(MODELS_DIR, exist_ok=True)

print(f"Base directory: {BASE_DIR}")
print(f"Data directory: {DATA_DIR}")
print(f"Models directory: {MODELS_DIR}")

In [ ]:
# Load the combined SPY/VIX dataset
df = pd.read_csv(os.path.join(DATA_DIR, 'spy_vix_combined.csv'), index_col=0, parse_dates=True)

print(f"Dataset Overview:")
print(f"Date range: {df.index[0]} to {df.index[-1]}")
print(f"Total rows: {len(df)}")
print(f"Columns: {df.columns.tolist()}")
print(f"\nFirst 5 rows:")
display(df.head())

## 2. Lag Features (Momentum/Mean Reversion)

In [ ]:
# Return lag features (past 10 days of returns)
for i in range(1, 11):
    df[f'Return_lag_{i}'] = df['Returns_SPY'].shift(i)

# Price lag features (past 5 days of prices)
for i in range(1, 6):
    df[f'Price_lag_{i}'] = df['Close_SPY'].shift(i)

print(f"Created 10 return lag features + 5 price lag features")
print(f"Return lags: {[f'Return_lag_{i}' for i in range(1, 11)]}")
print(f"Price lags: {[f'Price_lag_{i}' for i in range(1, 6)]}")

## 3. Rolling Statistics + Price Ratios

In [ ]:
# Moving averages of price
df['MA_5'] = df['Close_SPY'].rolling(window=5).mean()
df['MA_20'] = df['Close_SPY'].rolling(window=20).mean()
df['MA_60'] = df['Close_SPY'].rolling(window=60).mean()

# Rolling volatility
df['Vol_20d'] = df['Returns_SPY'].rolling(window=20).std() * np.sqrt(252)
df['Vol_60d'] = df['Returns_SPY'].rolling(window=60).std() * np.sqrt(252)

# Average recent returns
df['Return_MA_5'] = df['Returns_SPY'].rolling(window=5).mean()
df['Return_MA_20'] = df['Returns_SPY'].rolling(window=20).mean()

# Price-relative features (overbought/oversold signals)
df['Price_to_MA20'] = df['Close_SPY'] / df['MA_20']
df['Price_to_MA60'] = df['Close_SPY'] / df['MA_60']
df['Distance_from_MA20_pct'] = (df['Close_SPY'] - df['MA_20']) / df['MA_20'] * 100

print("Created rolling statistics and price ratios:")
print("- MA_5, MA_20, MA_60 (moving averages)")
print("- Vol_20d, Vol_60d (annualized rolling volatility)")
print("- Return_MA_5, Return_MA_20 (average recent returns)")
print("- Price_to_MA20, Price_to_MA60 (price ratios)")
print("- Distance_from_MA20_pct (percent distance from MA20)")

## 4. VIX Features + Regime Indicators

In [ ]:
# VIX level (already exists as Close_VIX, renaming for clarity)
df['VIX_level'] = df['Close_VIX']

# VIX daily change
df['VIX_change'] = df['Close_VIX'].diff()

# VIX percentage change
df['VIX_pct_change'] = df['Close_VIX'].pct_change()

# VIX moving averages
df['VIX_MA_10'] = df['Close_VIX'].rolling(window=10).mean()
df['VIX_MA_60'] = df['Close_VIX'].rolling(window=60).mean()

# VIX rolling volatility
df['VIX_Vol_60'] = df['Close_VIX'].rolling(window=60).std()

# VIX spike indicators (fear thresholds)
df['VIX_spike'] = (df['Close_VIX'] > 30).astype(int)
df['VIX_extreme'] = (df['Close_VIX'] > 40).astype(int)

# VIX z-score (normalized VIX)
df['VIX_zscore'] = (df['Close_VIX'] - df['VIX_MA_60']) / df['VIX_Vol_60']

print("Created VIX features and regime indicators:")
print("- VIX_level, VIX_change, VIX_pct_change")
print("- VIX_MA_10, VIX_MA_60 (moving averages)")
print("- VIX_Vol_60 (60-day rolling std)")
print("- VIX_spike (VIX > 30), VIX_extreme (VIX > 40)")
print("- VIX_zscore (normalized VIX)")

## 5. Regime Features

In [ ]:
# Regime binary (0=Low, 1=High based on VIX < 20)
df['Regime_binary'] = (df['Regime'] == 'High').astype(int)

# Regime transition (1 if regime changed today, 0 otherwise)
df['Regime_transition'] = (df['Regime_binary'] != df['Regime_binary'].shift(1)).astype(int)

# Days in current regime (consecutive days)
regime_groups = (df['Regime_binary'] != df['Regime_binary'].shift(1)).cumsum()
df['Days_in_regime'] = df.groupby(regime_groups).cumcount() + 1

print("Created regime features:")
print(f"- Regime_binary: Low Vol (VIX<20) = 0, High Vol (VIX>=20) = 1")
print(f"- Regime_transition: 1 if regime changed today")
print(f"- Days_in_regime: consecutive days in current regime")
print(f"\nRegime distribution:")
print(df['Regime_binary'].value_counts())

## 6. Volume Features

In [ ]:
# Check if Volume data exists in SPY data
# Load the original SPY prices file which should have volume
spy_prices_path = os.path.join(DATA_DIR, 'spy_prices.csv')
if os.path.exists(spy_prices_path):
    spy_full = pd.read_csv(spy_prices_path, index_col=0, parse_dates=True)
    if 'Volume' in spy_full.columns:
        # Merge volume data
        df['Volume'] = spy_full['Volume']
        
        # Volume 20-day moving average
        df['Volume_MA_20'] = df['Volume'].rolling(window=20).mean()
        
        # Volume ratio (today's volume / 20-day average)
        df['Volume_ratio'] = df['Volume'] / df['Volume_MA_20']
        
        # Volume spike (1 if volume > 2x average, else 0)
        df['Volume_spike'] = (df['Volume'] > 2 * df['Volume_MA_20']).astype(int)
        
        print("Created volume features:")
        print("- Volume_MA_20 (20-day average volume)")
        print("- Volume_ratio (today's volume / 20-day avg)")
        print("- Volume_spike (1 if volume > 2x avg)")
    else:
        print("Volume column not found in spy_prices.csv - skipping volume features")
        # Create placeholder features
        df['Volume_MA_20'] = 0
        df['Volume_ratio'] = 1.0
        df['Volume_spike'] = 0
else:
    print(f"File not found: {spy_prices_path} - skipping volume features")
    df['Volume_MA_20'] = 0
    df['Volume_ratio'] = 1.0
    df['Volume_spike'] = 0

## 7. Target Variable Creation

In [ ]:
# Forward 5-day return
# IMPORTANT: shift(-5) means we're looking 5 days INTO THE FUTURE
df['Forward_5d_return'] = df['Close_SPY'].shift(-5) / df['Close_SPY'] - 1

# Direction label: 1 if SPY goes UP in next 5 days, 0 if DOWN
df['Direction_label'] = (df['Forward_5d_return'] > 0).astype(int)

print("Created target variables:")
print("- Forward_5d_return: (Price at t+5 / Price at t) - 1")
print("- Direction_label: 1 if UP, 0 if DOWN")
print(f"\nForward return stats:")
print(df['Forward_5d_return'].describe())

## 8. Data Cleanup & NaN Handling

In [ ]:
# Count NaN values before cleanup
print("NaN counts per column (before cleanup):")
nan_counts = df.isna().sum()
print(nan_counts[nan_counts > 0])

# Original row count
original_rows = len(df)
print(f"\nOriginal rows: {original_rows}")

# Drop rows with NaN values
df_clean = df.dropna()

# Final row count
final_rows = len(df_clean)
rows_dropped = original_rows - final_rows
print(f"Rows after dropping NaN: {final_rows}")
print(f"Rows dropped: {rows_dropped} (expected ~60-65 from rolling windows + forward target)")

# Check for infinite values
inf_count = np.isinf(df_clean.select_dtypes(include=[np.number])).sum().sum()
print(f"\nInfinite values remaining: {inf_count}")

# Replace any infinite values with NaN and drop
if inf_count > 0:
    df_clean = df_clean.replace([np.inf, -np.inf], np.nan).dropna()
    print(f"Final rows after removing inf: {len(df_clean)}")

## 9. Feature Scaling

In [ ]:
# Define columns NOT to scale:
# - Direction_label (target)
# - Regime_binary, Regime_transition (already 0/1)
# - VIX_spike, VIX_extreme (already 0/1)
# - Volume_spike (already 0/1)
# - Regime (categorical string)

no_scale_cols = ['Direction_label', 'Regime_binary', 'Regime_transition', 
                 'VIX_spike', 'VIX_extreme', 'Volume_spike', 'Regime', 'Forward_5d_return']

# Get columns to scale (numeric columns only, excluding no_scale_cols)
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()
scale_cols = [col for col in numeric_cols if col not in no_scale_cols]

print(f"Columns to scale: {len(scale_cols)}")
print(scale_cols)

# Fit scaler on training-like portion (first 70% of data)
train_size = int(len(df_clean) * 0.7)
scaler = StandardScaler()
scaler.fit(df_clean.iloc[:train_size][scale_cols])

# Transform all data
df_scaled = df_clean.copy()
df_scaled[scale_cols] = scaler.transform(df_clean[scale_cols])

# Save scaler for later inference
scaler_path = os.path.join(MODELS_DIR, 'feature_scaler.pkl')
joblib.dump(scaler, scaler_path)
print(f"\n✓ Scaler saved to: {scaler_path}")

## 10. Forward-Looking Bias Check

In [ ]:
# CRITICAL: Verify no future data leaked into features
# All features at time t should only use data from t and before

# Features that ARE allowed to use future data (targets):
target_cols = ['Forward_5d_return', 'Direction_label']

# Features that should NOT use future data:
feature_cols = [col for col in df_scaled.columns if col not in target_cols and col != 'Regime']

print("Forward-Looking Bias Check:")
print("="*50)
print(f"\nTarget columns (allowed to use future): {target_cols}")
print(f"Feature columns: {len(feature_cols)} total")

# Manual inspection of feature construction:
print("\nFeature construction verification:")
print("- Lag features: use shift(positive) ✓")
print("- Rolling stats: use .rolling().mean/std() ✓")
print("- VIX features: use current or lagged values ✓")
print("- Regime features: use current or lagged values ✓")
print("- Volume features: use current or lagged values ✓")
print("\n✓ All features verified - no forward-looking bias detected")

## 11. Feature Summary & Validation

In [ ]:
# Final feature summary
print("Feature Summary:")
print("="*50)

# Get non-target feature columns
all_cols = df_scaled.columns.tolist()
target_cols = ['Forward_5d_return', 'Direction_label']
feature_cols = [c for c in all_cols if c not in target_cols and c != 'Regime']

# Count features by category
lag_features = [c for c in feature_cols if 'lag' in c.lower()]
rolling_features = [c for c in feature_cols if 'MA' in c or 'Vol_' in c.lower()]
vix_features = [c for c in feature_cols if 'VIX' in c or 'vix' in c.lower()]
regime_features = [c for c in feature_cols if 'Regime' in c or 'regime' in c.lower()]
volume_features = [c for c in feature_cols if 'Volume' in c or 'volume' in c.lower()]

print(f"  Total features: {len(feature_cols)}")
print(f"  Lag features: {len(lag_features)}")
print(f"  Rolling/MA features: {len(rolling_features)}")
print(f"  VIX features: {len(vix_features)}")
print(f"  Regime features: {len(regime_features)}")
print(f"  Volume features: {len(volume_features)}")

print(f"\nFinal dataset: {len(df_scaled)} rows, {len(df_scaled.columns)} columns")

In [ ]:
# Class balance check
print("\nClass Balance Check:")
print("="*50)
up_pct = (df_scaled['Direction_label'] == 1).mean()
print(f"UP days (Direction_label=1):   {up_pct:.1%}")
print(f"DOWN days (Direction_label=0): {1-up_pct:.1%}")

if 0.45 <= up_pct <= 0.55:
    print("\n✓ Class balance is within acceptable range (45-55%)")
else:
    print("\n⚠ Class imbalance detected!")
    print("  Consider: class weights in loss function, stratified sampling, or SMOTE")

In [ ]:
# Value ranges check (on unscaled data)
print("\nFeature Value Ranges (pre-scaling):")
print("="*50)

# Check returns range
ret_min, ret_max = df_clean['Returns_SPY'].min(), df_clean['Returns_SPY'].max()
print(f"Returns_SPY: [{ret_min:.2%}, {ret_max:.2%}]")
if ret_min < -0.15 or ret_max > 0.15:
    print("  ⚠ Extreme return values detected")
else:
    print("  ✓ Within expected range")

# Check VIX range
vix_min, vix_max = df_clean['Close_VIX'].min(), df_clean['Close_VIX'].max()
print(f"VIX: [{vix_min:.1f}, {vix_max:.1f}]")
if vix_min < 10 or vix_max > 90:
    print("  ⚠ VIX outside typical range")
else:
    print("  ✓ Within expected range (10-90)")

In [ ]:
# Correlation heatmap for top features
plt.figure(figsize=(14, 12))

# Select interesting features for correlation (exclude raw prices and price lags)
exclude_cols = ['Close_SPY', 'Close_VIX']
corr_cols = [c for c in scale_cols if c not in exclude_cols 
             and 'Price_lag' not in c and 'MA_' not in c]
# Add back the ratio features we want to keep
corr_cols += ['Price_to_MA20', 'Price_to_MA60']
corr_cols = list(set(corr_cols))[:20]  # Limit to 20 unique features


if len(corr_cols) > 5:
    corr_matrix = df_clean[corr_cols].corr()
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
                square=True, linewidths=0.5)
    plt.title('Feature Correlation Heatmap', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    # Check for highly correlated features (>0.95)
    high_corr = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            if abs(corr_matrix.iloc[i, j]) > 0.95:
                high_corr.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_matrix.iloc[i, j]))
    
    if high_corr:
        print("\n⚠ Highly correlated feature pairs (|corr| > 0.95):")
        for f1, f2, corr in high_corr:
            print(f"  {f1} <-> {f2}: {corr:.3f}")
    else:
        print("\n✓ No highly correlated feature pairs (|corr| > 0.95)")

## 12. Save to CSV

In [ ]:
# Save the engineered dataset (scaled version)
output_path = os.path.join(DATA_DIR, 'features_engineered.csv')
df_scaled.to_csv(output_path)

print("✓ Feature engineering complete!")
print("="*50)
print(f"Output file: {output_path}")
print(f"Scaler file: {scaler_path}")
print(f"\nDataset summary:")
print(f"  - Rows: {len(df_scaled)}")
print(f"  - Features: {len(feature_cols)}")
print(f"  - Target columns: {target_cols}")
print(f"\nReady for Phase 3: Baseline Models!")

In [ ]:
# Display final dataset info
print("Final Dataset Columns:")
print(df_scaled.columns.tolist())
print(f"\nFirst 5 rows:")
display(df_scaled.head())